<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Type I Error, Type II Error, and Power Analysis

_Authors: Kiefer Katovich (SF)_

---

### Learning Objectives
- Apply an understanding of statistical hypothesis testing within the context of split testing.
- Apply the chi-squared test of independence to "winner" a split test.
- Understand the relationship between p-values, alpha thresholds, and statistical significance.
- Understand the difference between a type I error, statistical power, and a type II error.
- Visualize the interaction of `alpha` and `power`.
- Understand how the components of experimental design interact.
- Learn how to calculate the statistical power of a test.
- Learn how to calculate the required sample size of a test.
- Visualize the required sample size of a test, given its other requirements.


### Lesson Guide
- [Introduction to A/B Testing](#ab-testing)
- [Split Tests are Hypothesis Tests](#ab-hypothesis)
- [Chi-Squared Test of Independence](#chisq)
- [Statistical Significance, P-Values, The Alpha Threshold, and Type I Errors](#significance)
- [Type II Errors and Statistical Power](#type2-power)
- [Visualizing the Interaction Between `alpha` and `power`](#power-visual)
- [Power Analysis and Sample Size Calculation](#power-analysis)
- [Calculating the Required Sample Size Visually](#visual-equation)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='ab-testing'></a>

### Introduction to A/B testing

---

You may have heard the term "A/B testing," or "split testing," before. Simply put, a split test is an experiment that tests different versions of your product with your users. Using these results, you as the data scientist will statistically analyze the experiment and determine a "winner" according to a pre-defined metric. 

**Example: Selling dog collars**

Picture this: You work for a startup that sells dog collars. Your web development team has constructed a prototype for a new "landing page" on the website (a landing page is the first page users reach when visiting a site). The designers are not sure whether a picture of a black lab wearing the collar or a golden retriever wearing the collar will have more of an impact on the click-through rate (the proportion of users who continue on to the the rest of the website). 

The team decides to run an A/B test to quantitatively evaluate which picture to choose.
- **Arm A** is the version of the landing page with the black lab.
- **Arm B** is the version of the landing page with the golden retriever.

For two weeks, users will be directed at random to one of the two landing pages with equal probability. At the end of this period, the click-through rates of each arm will be compared and one of the two will be "winnered."

Desiging and evaluating A/B tests like this one is one of the most common tasks a data scientist will be asked to perform.

**Below are the click-through rates per arm, measured at the end of two weeks.**

In [1]:
A_wins = 113
A_loss = 87
B_wins = 87
B_loss = 103

<a id='ab-hypothesis'></a>

### Split Tests Are Hypothesis Tests

---

Despite the business jargon, **split tests are just experiments to test hypotheses.** Using the scenario above, we can frame the null hypothesis like so:

> **H0:** The difference in click-through rates between arms is 0.

The alternative hypothesis would be:

> **H1:** The difference in click-through rates between arms is not 0.

It's important that the users sent to each arm are selected at random. If user assignments are affected by external factors — such as whether they are viewing the site on web or mobile browsers — then the arms have **selection bias**.

**What is the problem with choosing a picture if users were not randomly assigned?**

<a id='chisq'></a>

### The $\chi^2$ (Chi-Squared) Test of Independence

---

A popular Frequentist method for evaluating A/B tests is the $\chi^2$ test of independence. The $\chi^2$ test of independence is appropriate when you have categorical data and want to evaluate whether or not two groups are significantly different. 

Click-through rate can be thought of as binary categorical data: A user either clicked through (1) or did not (0). 

"Independence" refers to whether or not the outcome for the groups (the click-through rate) is independent of group assignment. Independence would mean that there is no relationship between the dog picture and the click-through rate. 

You can conduct the $\chi^2$ test manually using what is known as a contingency table. For a detailed overview of the procedure, [this site](https://onlinecourses.science.psu.edu/stat500/node/56) is a good resource. In this course, we will use Python instead of manual calculation. That being said, it is important to address the formula for the $\chi^2$ statistic:

### $$ \chi^2 = \sum_{i=1}^{cells} \frac{(O_i - E_i)^2}{E_i} $$

Where: 

- `$cells$` refers to the number of cells in the contingency table.
- `$O$` are the observed values (frequencies).
- `$E$` are the *expected* frequencies under perfect independence. 

**Using `stats.chi2_contingency`, calculate the `$\chi^2$` statistic and the associated p-value for our split test.**

In [4]:
table = np.array([[A_wins, A_loss],
                  [B_wins, B_loss]])

results = stats.chi2_contingency(table)
chi2 = results[0]
pvalue = results[1]
print chi2, pvalue

4.05546658587 0.0440285500395


**Explain what the p-value means in the context of our split test.**

_We have a p-value of about 0.044. This means that there is a 4.4% chance that we observed the difference in click-through rates between arms because of randomness in our sample when, in fact, there is no difference in the overall population. (That is, if everyone in the world were to visit our site)._

**Which arm is the "winner"? Should you choose to accept it as such? By how much?**

In [5]:
# Arm A has the higher click-through rate and would be the winner if this level
# of significance were sufficient.

A_rate = float(A_wins)/(A_wins+A_loss)
B_rate = float(B_wins)/(B_wins+B_loss)
A_vs_B = A_rate - B_rate
print A_rate, B_rate
print A_vs_B
print A_rate/B_rate

0.565 0.457894736842
0.107105263158
1.23390804598


<a id='significance'></a>

### Statistical Significance, P-Values, The Alpha Threshold, and Type I Errors

---

The split test has concluded and you have performed your statistical analysis of the results.
- **Arm A (the black lab picture) has a 23% higher click-through rate than Arm B (the golden retriever picture)**.
- **The p-value of our test was 0.044**.

Should we accept that the black lab picture is in fact more effective? Do we believe that the difference is real?

**Statistical Significance**

It's common to see Frequentist tests reported as "statistically significant with p < 0.05" or "with p < 0.01." So, what does it mean for a test to be statistically significant? On the surface, these statements are simply saying that the calculated p-value is less than a specific value. The values of 0.05 and 0.01 are common in academic research but also arbitrary. 

What does having "p < 0.05" specifically mean?

> **p < 0.05**: In hypothetical repetitions of this experiment with the same sample size, fewer than 5% of the experiments would have measured a difference between arms at least this extreme _by chance_.

The same goes for "p < 0.01." Here, however, it's framed in the context of null and alternative hypotheses:

> **p < 0.01**: There is less than a 1% chance of accepting the alternative hypothesis when the null hypothesis is in fact true. 

---

**Type I Errors and the Alpha Threshold ($\alpha$**)

As rigorous researchers, we would set a threshold for how likely we are to falsely accept the alternative hypothesis prior to running our experiment. This chance is known as a **type I error**.

Type I errors are directly related to the p-value. 

> **The p-value represents the risk of encountering a type I error, given the sample size and measured effect.**

It is important to set thresholds for type I errors before experiments begin. This prevents us from arbitrarily deciding whether or not we will accept an alternative hypothesis after we see a p-value.

The threshold we set for the type I error rate is denoted with `alpha`. For example:

- $\alpha$ = 0.05 corresponds to a "p < 0.05" significance level.
- $\alpha$ = 0.01 corresponds to a "p < 0.01" significance level.

Furthermore, `alpha` is equivalent to `1.0 - confidence`. A 95% confidence interval, for example, is associated with a type I error threshold of "$\alpha$ = 0.05."

**A Side Note on P-Value Thresholds:** 

"p < 0.01" is historically considered a "conservative" significance threshold. But it's actually not very conservative at all. 

This is, at worst, a 1/100 chance that an alternative hypothesis will be accepted when the result is in fact null. Now, think about all of the papers written and experiments run that have used a threshold of "p < 0.01" to validate their findings.
